In [ ]:
import SimpleITK as sitk
import matplotlib.pyplot as plt

# Read MR image
img = sitk.ReadImage("/home/abigail/synthetic_CT/910/MR_T1_TFE_1.5mm.nii")

smoothed = sitk.CurvatureFlow(img, timeStep=0.125, numberOfIterations=5)

# Threshold (adjust lowerThreshold based on your image intensity)
thresholded = sitk.BinaryThreshold(smoothed, lowerThreshold=1, upperThreshold=1000, insideValue=1, outsideValue=0)

# Fill holes
mask = sitk.BinaryFillhole(thresholded)

# Convert to NumPy for visualization
mask_array = sitk.GetArrayViewFromImage(mask)


In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Read mask
mask = sitk.ReadImage("/home/abigail/Synthrad2025_Task_1/Dataset/HN/910/MASK.mha")
mask_array = sitk.GetArrayViewFromImage(mask)  # [z, y, x]

# Create figure and initial display
fig, ax = plt.subplots(figsize=(6,6))
im = ax.imshow(mask_array[mask_array.shape[0]//2], cmap='gray')
ax.axis('off')
plt.close(fig)  # Avoid duplicate display in Jupyter

# Function to update image
def update_slice(plane, slice_idx):
    if plane == 'axial':
        im.set_data(mask_array[slice_idx])
    elif plane == 'coronal':
        im.set_data(mask_array[:, slice_idx, :])
    elif plane == 'sagittal':
        im.set_data(mask_array[:, :, slice_idx])
    ax.set_title(f"{plane} slice {slice_idx}")
    fig.canvas.draw_idle()
    display(fig)

# Widgets
plane_dropdown = widgets.Dropdown(options=['axial','coronal','sagittal'], value='axial')
slice_slider = widgets.IntSlider(min=0, max=mask_array.shape[0]-1, value=mask_array.shape[0]//2, description='Slice')

def on_plane_change(change):
    p = change['new']
    if p == 'axial':
        slice_slider.max = mask_array.shape[0]-1
    elif p == 'coronal':
        slice_slider.max = mask_array.shape[1]-1
    elif p == 'sagittal':
        slice_slider.max = mask_array.shape[2]-1
    slice_slider.value = slice_slider.max // 2

plane_dropdown.observe(on_plane_change, names='value')

widgets.interact(update_slice, plane=plane_dropdown, slice_idx=slice_slider)
